# Import


In [2]:
from PIL import Image
from glob import glob
from tqdm import tqdm_notebook
from collections import defaultdict
from sklearn import preprocessing
from xml.dom import minidom
import xml.etree.ElementTree as ET

# from data_aug.data_aug import *
# from data_aug.bbox_util import *

import matplotlib.pyplot as plt
import pandas as pd
import time
import numpy as np
import pickle as pkl
import cv2
import json
import os
import re

# Set path

In [2]:
path_2017 = "/home/ubunt/M2Det_data/Hynix data/2017_2018_img/"
whole_csv_path = "/home/ubunt/M2Det_data/Python script/"
af_path_2017 = "/home/ubunt/M2Det_data/Hynix data/2017_2018_img_af/"
image_path_2017 = af_path_2017 + "*.bmp"
csv_path_2017 = path_2017 + "*.csv"
cropped_path_2017 = "/home/ubunt/M2Det_data/Hynix data/2017_2018_img_cropped/"
image_path_recent = "/home/ubunt/M2Det_data/Hynix data/labeled_imgs_recent/"

# Recent data

In [3]:
# 특정 이미지 확인
def select_image_recent(img_path, file_name) :
    image_path = [x for x in sorted(img_path) if file_name in x][0]
    image = Image.open(image_path)
    image_np = np.array(image)
    return image, image_np

In [4]:
def check_key(file_name, df) :
    output = df.loc[df['IMG_NAME'] == file_name + ".jpg", 'ROW_UNIQUE_KEY'].tolist()
    #print(df.loc[df['IMG_NAME'] == file_name + ".jpg", 'ROW_UNIQUE_KEY'])
    return output

In [10]:
def make_two_data(whole_csv, margin) :
    cropped_data, bbox_data = whole_csv.copy(), whole_csv.copy()
    cropped_data['P_X1'] = [x - margin for x in cropped_data['P_X1']]
    cropped_data['P_X2'] = [x + margin for x in cropped_data['P_X2']]
    cropped_data['P_Y1'] = [x - margin for x in cropped_data['P_Y1']]
    cropped_data['P_Y2'] = [x + margin for x in cropped_data['P_Y2']]
    
    bbox_data['P_X1'] = [y - x for x,y in zip(cropped_data['P_X1'], bbox_data['P_X1'])]
    bbox_data['P_Y1'] = [y - x for x,y in zip(cropped_data['P_Y1'], bbox_data['P_Y1'])]
    bbox_data['P_X2'] = [x + y for x,y in zip(bbox_data['P_X1'], bbox_data['diff_x'])]
    bbox_data['P_Y2'] = [x + y for x,y in zip(bbox_data['P_Y1'], bbox_data['diff_y'])]
    
    # range_x, range_y
    bbox_data['range_x'] = [y - x for x,y in zip(cropped_data['P_X1'], cropped_data['P_X2'])]
    bbox_data['range_y'] = [y - x for x,y in zip(cropped_data['P_Y1'], cropped_data['P_Y2'])]
    
    # label ratio
    bbox_data['label_x1'] = [int(np.round((x / y) * 320)) for x,y in zip(bbox_data['P_X1'], bbox_data['range_x'])]
    bbox_data['label_y1'] = [int(np.round((x / y) * 320)) for x,y in zip(bbox_data['P_Y1'], bbox_data['range_y'])]
    bbox_data['label_x2'] = [int(np.round((x / y) * 320)) for x,y in zip(bbox_data['P_X2'], bbox_data['range_x'])]
    bbox_data['label_y2'] = [int(np.round((x / y) * 320)) for x,y in zip(bbox_data['P_Y2'], bbox_data['range_y'])]
    
    return cropped_data, bbox_data

* load data

In [19]:
date_list = ["2019_08_18","2019_08_19","2019_08_20","2019_08_21"]
img_path = []

for date in date_list :
    csv_path = [x for x in glob(image_path_recent + date + "/*") if ".csv" in x]
    folder_path = [x for x in glob(image_path_recent + date + "/*") if ".csv" not in x]
    for path in folder_path :
        img_path.extend(glob(path+ "/*.jpg"))
    csv_date = pd.read_csv(csv_path[0])
    del csv_date['Unnamed: 0']
    if date == "2019_08_18" :
        whole_csv = csv_date.copy()
    else :
        whole_csv = whole_csv.append(csv_date)
whole_csv.index = [x for x in range(whole_csv.shape[0])]

In [20]:
select_list = [True if x != 0 else False for x in whole_csv['P_X1']]
whole_csv = whole_csv.loc[select_list,:]

In [21]:
whole_csv['diff_x'] = [y - x for x,y in zip(whole_csv['P_X1'], whole_csv['P_X2'])]
whole_csv['diff_y'] = [y - x for x,y in zip(whole_csv['P_Y1'], whole_csv['P_Y2'])]

In [22]:
whole_csv.head(3)

,DATE,LOT,IMG_NAME,ROW_UNIQUE_KEY,PART,R_X1,R_Y1,R_X2,R_Y2,P_X1,P_Y1,P_X2,P_Y2,DEFECT_CLASS,diff_x,diff_y
0,2019-08-18,2019_08_18_LOT_A5,2019_08_18_LOT_A5_K_01_41.jpg,2019_08_18_LOT_A5_K_01_41_0,Cap,624,298,630,300,622,280,633,301,fail,11,21
1,2019-08-18,2019_08_18_LOT_A5,2019_08_18_LOT_A5_K_00_25.jpg,2019_08_18_LOT_A5_K_00_25_0,Cap,381,1745,402,1756,381,1745,402,1756,fail,21,11
2,2019-08-18,2019_08_18_LOT_A5,2019_08_18_LOT_A5_K_01_09.jpg,2019_08_18_LOT_A5_K_01_09_0,Cap,582,2698,588,2701,582,2681,592,2703,fail,10,22


In [23]:
select_list = [True if x != "Tab" else False for x in whole_csv.PART]
whole_csv = whole_csv.loc[select_list,:]
whole_csv.index = [x for x in range(whole_csv.shape[0])]

In [24]:
len(whole_csv.loc[whole_csv.PART == "Res",:].IMG_NAME.unique().tolist())

193

In [25]:
whole_csv = pd.read_csv("whole_csv_correct_finish_res.csv")

In [38]:
cropped_data, bbox_data = make_two_data(whole_csv,8)

In [3]:
folder_name = "dataset"
wd = '/home/jeon/Desktop/k-lab/train_our_data_M2Det/M2Det/data/'
Annotations_path = wd+"{}/Annotations/".format(folder_name)
Image_save_path = wd+"{}/JPEGImages/".format(folder_name)
txt_save_path = wd+"{}/ImageSets/Main/".format(folder_name)
val_save_path = wd+"{}/test set/".format(folder_name)

#### Crop Image file

In [40]:
unique_name_set = bbox_data.IMG_NAME.unique().tolist()

In [5]:
for name in tqdm_notebook(unique_name_set) :
    image, image_np = select_image_recent(img_path, name)
    tmp_df = cropped_data.loc[cropped_data['IMG_NAME'] == name,['ROW_UNIQUE_KEY2','PART','P_X1',"P_Y1","P_X2","P_Y2"]]
    
    for idx2,key in enumerate(tmp_df.ROW_UNIQUE_KEY2) :
        crop_image = image.crop((tmp_df.loc[tmp_df['ROW_UNIQUE_KEY2'] == key,'P_X1'].iloc[0],
                                tmp_df.loc[tmp_df['ROW_UNIQUE_KEY2'] == key,'P_Y1'].iloc[0],
                                tmp_df.loc[tmp_df['ROW_UNIQUE_KEY2'] == key,'P_X2'].iloc[0],
                                tmp_df.loc[tmp_df['ROW_UNIQUE_KEY2'] == key,'P_Y2'].iloc[0],
                               ))
        resize_image = crop_image.resize((320,320))
        resize_image.save(Image_save_path + key + ".jpg")
        if "2019_08_18" in key or ("LL4" in tmp_df.PART.iloc[idx2]) :
            if not os.path.isdir(val_save_path + tmp_df.PART.iloc[0]) :
                os.mkdir(val_save_path + tmp_df.PART.iloc[idx2])
            resize_image.save(val_save_path + tmp_df.PART.iloc[idx2] + "/" + key + ".jpg")

NameError: name 'unique_name_set' is not defined

#### Annotation file

In [42]:
final_bbox = bbox_data.loc[:,['IMG_NAME','ROW_UNIQUE_KEY2','PART','label_x1','label_y1','label_x2','label_y2']]
final_bbox.index = [x for x in range(final_bbox.shape[0])]

In [43]:
unique_key_set = bbox_data.ROW_UNIQUE_KEY2.unique()

In [44]:
for key in tqdm_notebook(unique_key_set) :
    tmp_df = final_bbox.loc[final_bbox.ROW_UNIQUE_KEY2 == key, :]
    # root
    anootation = ET.Element('annotation')
    # folder
    folder = ET.SubElement(anootation, 'folder')
    folder.text = "Hynix_train"
    # filename
    filename = ET.SubElement(anootation, 'filename')
    # 2007_000027.jpg
    filename.text = key + ".jpg" # '2019_08_18_LOT_A5_K_00_41_4.jpg'
    # path
    path = ET.SubElement(anootation, 'path')
    # /my/path/GeneratedData_Train/000001.png
    path.text = "/home/yjchoi/data/{}/JPEGImages/".format(folder_name) + key + ".jpg" 
    # souce
    source = ET.SubElement(anootation, 'source')
    database = ET.SubElement(source, 'database')
    database.text = folder_name
    # size
    size = ET.SubElement(anootation, 'size')
    width = ET.SubElement(size, 'width')
    width.text = str(320)
    height = ET.SubElement(size, 'height')
    height.text= str(320)
    depth = ET.SubElement(size, 'depth')
    depth.text = str(3)
    # segmented
    segmented = ET.SubElement(anootation, 'segmented')
    segmented.text = str(0)
    for i in range(tmp_df.shape[0]) :
        # object
        object = ET.SubElement(anootation, 'object')
        name = ET.SubElement(object, 'name')
        name.text = tmp_df['PART'].iloc[i]
        pose = ET.SubElement(object, 'pose')
        pose.text = "Unspecified"
        truncated = ET.SubElement(object, 'truncated')
        truncated.text = str(0)
        difficult = ET.SubElement(object, 'difficult')
        difficult.text = str(0)
        occluded = ET.SubElement(object, 'occluded')
        occluded.text = str(0)
        bndbox = ET.SubElement(object, 'bndbox')
        xmin = ET.SubElement(bndbox, 'xmin')
        xmin.text = str(int(tmp_df['label_x1'].iloc[i]))
        xmax = ET.SubElement(bndbox, 'xmax')
        xmax.text = str(int(tmp_df['label_x2'].iloc[i]))
        ymin = ET.SubElement(bndbox, 'ymin')
        ymin.text = str(int(tmp_df['label_y1'].iloc[i]))
        ymax = ET.SubElement(bndbox, 'ymax')
        ymax.text = str(int(tmp_df['label_y2'].iloc[i]))

    # create a new XML file with the results
    mydata = ET.tostring(anootation)
    #myfile = open(Annotations_path + key + plus_name + ".xml", "w")
    myfile = open(Annotations_path + key + ".xml", "w")
    myfile.write(str(mydata)[2:(len(str(mydata))-1)])
    while True :
        if not myfile.closed :
            myfile.close()
        else :
            break

#### txt file

In [45]:
whole_train_txt = open(txt_save_path + "whole_train.txt", 'a')
for key in tqdm_notebook(unique_key_set) :
    whole_train_txt.write(key + "\n")
whole_train_txt.close()

In [46]:
unique_part_set = final_bbox.PART.tolist()

In [47]:
train_txt = open(txt_save_path + "train.txt", 'a')
for key,part in tqdm_notebook(zip(unique_key_set,unique_part_set)) :
    if part == "LL8" or "2019_08_18" not in key :
        train_txt.write(key + "\n")
train_txt.close()

In [48]:
val_txt = open(txt_save_path + "val.txt", 'a')
for key in tqdm_notebook(unique_key_set) :
    if "2019_08_18" in key :
        val_txt.write(key + "\n")
val_txt.close()